# Plotting Temporal changes

I will plot all heights regardless of locations

## Test

One track would be plotted as an example

### Setting

In [2]:
import glob
import os
import pandas as pd
from pathlib import Path
import h5py
import re
import numpy as np
import matplotlib.pyplot as plt
import datetime
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import statistics
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/download/')
## list them up and check them
files= list(data_home.glob('*.h5'))
## choose a file as an example

##### function of reading alt13
def alt13_to_df(filename, beam):    
    f = h5py.File(filename, 'r')
    f_beam = f[beam]
    lat = f_beam['segment_lat'][:]
    long = f_beam['segment_lon'][:]
    ws = f_beam['ht_water_surf'][:]
    ws_sd = f_beam['stdev_water_surf'][:]
    ws_err = f_beam['err_ht_water_surf'][:]
    ortho = f_beam['ht_ortho'][:]
    wd = f_beam['water_depth'][:]
    alt13_df = pd.DataFrame({'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                            'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                            'WaterD':wd})
    return alt13_df

### We set 'gt2l' as a beam of example for tracking
D_dict={}
error_count=0
for ff in files:
    try:
        D_dict[ff]=alt13_to_df(ff, 'gt2l')
    except KeyError as e:
        ##print(f'file {ff} encountered error {e}')
        error_count += 1
        
### bounds of Tonle Sap Lake
sp_ex = [103.643, 104.667, 12.375, 13.287]

### Plot of one track

##### I need to make a col and draw it repetitively

Function for record the beam, date and RGT, as well as other information.

In [4]:
### I made a function for stacking the information
def alt13_to_df_beams(filename):    
    f = h5py.File(filename, 'r')
    rgt = str(filename).split("_")[5][0:4]
    cycle = int(str(filename).split("_")[5][4:6])
    version = int(str(filename).split("_")[5][6:8])
    ymd = str(filename).split("_")[4][0:8]
    ymd_trans = datetime.datetime(int(ymd[0:4]),int(ymd[4:6]),int(ymd[6:8]))
    date = ymd_trans.strftime("%Y-%m-%d")
    beam_lst = list(f)[2:-1]
    alt13_df = pd.DataFrame()
    for beam in beam_lst:
        f_beam = f[beam]
        lat = f_beam['segment_lat'][:]
        long = f_beam['segment_lon'][:]
        ws = f_beam['ht_water_surf'][:]
        ws_sd = f_beam['stdev_water_surf'][:]
        ws_err = f_beam['err_ht_water_surf'][:]
        ortho = f_beam['ht_ortho'][:]
        wd = f_beam['water_depth'][:]
        df_beam = pd.DataFrame({'Beam': beam ,'RGT':rgt,'Cycle': cycle, 'Date':date, 'Date_num':int(ymd),'Ver.':version,
                                 'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                                'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                                'WaterD':wd})
        alt13_df = alt13_df.append(df_beam, ignore_index = True)
        
    return alt13_df


In [ ]:
test_beams = alt13_to_df_beams(files[37])

In [ ]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(test_beams['Date'],test_beams['SurfaceH'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

## Plot of all tracks with all beams

In [5]:
a_tracks = pd.DataFrame()
for ff in files:
    a_tracks = a_tracks.append(alt13_to_df_beams(ff), ignore_index = True)
print('done')

done


In [6]:
#### Sorting to draw graph
a_tracks.sort_values(by=['Date_num'], inplace=True)

In [7]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.plot(a_tracks['Date'],a_tracks['SurfaceH'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot the tracks by grouping their locations 

#### Map all tracks with labels

Drawing a just track first.

In [8]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(sp_ex, crs=ccrs.PlateCarree())
plt.scatter(a_tracks['Longitude'], a_tracks['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 10)
plt.title("Tracks on TSL")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Tracks on TSL')

Let's plot water levels according to cycles.

Before it, I will plot the third track, and see the temporal gaps

In [9]:
#### Select the 3rd track
track_3rd = a_tracks.loc[a_tracks['Cycle'] == 3]

In [10]:
#### Mapping
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(sp_ex, crs=ccrs.PlateCarree())
plt.scatter(track_3rd['Longitude'], track_3rd['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 10)
plt.title("Tracks on TSL")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Tracks on TSL')

In [ ]:
print(track_3rd['Date_num'].max()-track_3rd['Date_num'].min())

It is about 201 day. Too long to see the water changes.

I will see some papers and see how they make a graph.

Here are my plans to do today.
1. Plot outliers
2. Add averaged points of the day
3. See the relationship of water level with extent
4. Newly apply to Bugon Res.

## Plot outliers and averaged heights

Outliers
IQR = q0.75 − q0.25
LSHoutlier > q0.75 + 1.5 ∗ IQR or LSHoutlier < q0.25 − 1.5 ∗ IQR

I need to do it by beams, so I need to modify the function

In [ ]:
a_tracks.head()

#### Function for removing outliers

In [11]:
### outliers
def out_rmv(df,var):
    ### lower (25%)
    lwq = df[var].quantile(0.25)
    ### upper (75%)
    hwq = df[var].quantile(0.75)

    ### IQR
    iqr = hwq - lwq

    lw_out = lwq-1.5*iqr
    hw_out = hwq+1.5*iqr

    ###LSHoutlier > q0.75 + 1.5 ∗ IQR or LSHoutlier < q0.25 − 1.5 ∗ IQR
    return df.loc[(df[var] >= lw_out) & (df[var] <= hw_out)]

#### Function for stacking DB without outliers

In [12]:
### I made a function for stacking the information
def alt13_to_df_beams_out(filename):    
    f = h5py.File(filename, 'r')
    rgt = str(filename).split("_")[5][0:4]
    cycle = int(str(filename).split("_")[5][4:6])
    version = int(str(filename).split("_")[5][6:8])
    ymd = str(filename).split("_")[4][0:8]
    ymd_trans = datetime.datetime(int(ymd[0:4]),int(ymd[4:6]),int(ymd[6:8]))
    date = ymd_trans.strftime("%Y-%m-%d")
    beam_lst = list(f)[2:-1]
    alt13_df = pd.DataFrame()
    for beam in beam_lst:
        f_beam = f[beam]
        lat = f_beam['segment_lat'][:]
        long = f_beam['segment_lon'][:]
        ws = f_beam['ht_water_surf'][:]
        ws_sd = f_beam['stdev_water_surf'][:]
        ws_err = f_beam['err_ht_water_surf'][:]
        ortho = f_beam['ht_ortho'][:]
        wd = f_beam['water_depth'][:]
        df_beam = pd.DataFrame({'Beam': beam ,'RGT':rgt,'Cycle': cycle, 'Date':date, 'Date_num':int(ymd),'Ver.':version,
                                 'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                                'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                                'WaterD':wd})
        alt13_df = alt13_df.append(df_beam, ignore_index = True)
    
    alt13_df_out = pd.DataFrame()
    for beam_type in beam_lst:
        df_select = alt13_df.loc[alt13_df['Beam']==beam_type]
        alt13_df_out = alt13_df_out.append(out_rmv(df_select,'SurfaceH'), ignore_index = True)
        
    return alt13_df_out

In [13]:
### I made a function for stacking the information
def alt13_a_out(db,var2):
    alt13_df = pd.DataFrame()
    date_list = db['Date_num'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date_num']==prd]
        beam_lst = db['Beam'].unique()
        for beam_type in beam_lst:
            df_select = s_db.loc[s_db['Beam']==beam_type]
            alt13_df = alt13_df.append(out_rmv(df_select,var2), ignore_index = True)
    return alt13_df

In [14]:
a_tracks_out=alt13_a_out(a_tracks,'SurfaceH')

Compared the numbers

In [ ]:
len(a_tracks_out)

In [ ]:
len(a_tracks)

#### Function for making the averaged heights

In [ ]:
print(statistics.mean(test['SurfaceH']))

In [ ]:
print(test['Date'].unique()[0])

In [15]:
### I made a function for averaginf the height
def alt13_mean(db,var2):
    mean_df_list = pd.DataFrame()
    date_list = db['Date_num'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date_num']==prd]
        av_height = statistics.mean(s_db[var2])
        mean_df = pd.DataFrame({'Av_level':av_height,'Date':s_db['Date'].unique(), 'Date_num':s_db['Date_num'].unique()})
        mean_df_list = mean_df_list.append(mean_df, ignore_index = True)
    return mean_df_list

In [16]:
av_h = alt13_mean(a_tracks_out,'SurfaceH')

In [17]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.plot(av_h['Date'],av_h['Av_level'],'+',markersize=4, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot segments and segments without outliers

In [85]:
fig=plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.set_ylim([-20,0])
ax.plot(a_tracks['Date'],a_tracks['SurfaceH'],'.', color = 'orange', markersize=0.25, label='all segements')
ax.plot(a_tracks_out['Date'],a_tracks_out['SurfaceH'],'.', color = 'green',markersize=0.4, label='Without outliers')
ax.plot(av_h['Date'],av_h['Av_level'],'+', color = 'red',markersize=3, label='Averaged')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
a_tracks['Date'].unique()

array(['2018-10-28', '2018-11-01', '2018-11-02', '2018-11-30',
       '2018-12-01', '2018-12-30', '2019-01-03', '2019-01-27',
       '2019-01-31', '2019-02-01', '2019-03-01', '2019-03-30',
       '2019-04-03', '2019-05-02', '2019-05-31', '2019-07-28',
       '2019-08-01', '2019-08-29', '2019-08-30', '2019-09-28',
       '2019-10-27', '2019-10-31', '2019-11-28', '2019-11-29',
       '2019-12-27', '2019-12-28', '2020-01-01', '2020-01-25',
       '2020-01-29', '2020-01-30', '2020-02-23', '2020-02-27',
       '2020-02-28', '2020-03-27', '2020-03-28'], dtype=object)

### Load changes of water extents of the TSL

I used GEE to extract the water level changes using MODIS. I made monthly water mask for the cloud-free images and extracted the extents. For filling some gaps due to the clouds, images of one and two years before are used.

In [22]:
pwd

'/home/jovyan/ICESat_water_level/extraction'

In [50]:
we = pd.read_csv("./data/200706_water_TSL.csv")

In [51]:
##Create the col of readable date
date_lst = []
for dates in we['Date']:
    ymd_trans = datetime.datetime(int(str(dates)[0:4]),int(str(dates)[4:6]),int(str(dates)[6:8]))
    date_trans = ymd_trans.strftime("%Y-%m-%d")
    date_lst.append(date_trans)
we.insert(2, "Date2", date_lst, True) 

In [55]:
## remove 190801 due to spurring values
we = we.drop(11)
print(we)

    system:index         Area       Date2  id      Date
0              0  3370.644055  2018-09-01   0  20180901
1              1  3703.477608  2018-10-01   1  20181001
2              2  2012.177456  2018-11-01   2  20181101
3              3  1836.499641  2018-12-01   3  20181201
4              4  2288.920053  2019-01-01   4  20190101
5              5  2287.976726  2019-02-01   5  20190201
6              6  2124.563827  2019-03-01   6  20190301
7              7  2214.760226  2019-04-01   7  20190401
8              8  2559.180276  2019-05-01   8  20190501
9              9  2575.179994  2019-06-01   9  20190601
10            10  2294.466438  2019-07-01  10  20190701
12            12  3291.718026  2019-09-01  12  20190901
13            13  2508.601048  2019-10-01  13  20191001
14            14  2715.205733  2019-11-01  14  20191101
15            15  2334.157314  2019-12-01  15  20191201
16            16  2348.783952  2020-01-01  16  20200101
17            17  2869.605395  2020-02-01  17  2

In [78]:
### plot
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))

x = we['Date2']
y = we['Area']
plt.plot(x,y, linewidth=1,color = '#45b6fe')
ax.plot(x,y,'o',markersize=4, label='MODIS-Driven Extents',color = '#1c4966')

h_leg=ax.legend()
plt.title('Water Extents')
ax.set_xlabel('Date')
ax.set_ylabel('Water Extents, km^2')
plt.show()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
##3 Save heights
av_h.to_csv("./data/200706_waterlevel_TSL.csv")

## Install GEE to process the water extraction

It does not work well. 

In [ ]:
pip install earthengine-api

In [ ]:
pip install earthengine-api --upgrade

In [1]:
import ee

In [2]:
ee.Authenticate()

Enter verification code:  4/1gEBxgk5Mq8dmKFQrrkfsFNF4ilSoLvEQ06SuhE2M0u9rmwFn7paSps



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
ndwi = ee.ImageCollection("LANDSAT/LC08/C01/T1_8DAY_NDWI").filterDate('2018-01-01','2020-07-01')

In [17]:
list(ndwi.getInfo())[4]

'properties'